In [1]:
import numpy as np
import pandas as pd

In [2]:
from simpletransformers.classification import ClassificationModel

In [3]:
a=pd.read_excel("Projem.xlsx")
a

,kategori,metin
0,Bilim Ve Teknoloji,Fransa Ulusal Arkeolojik Araştırma Enstitüsü’n...
1,Bilim Ve Teknoloji,"Baykar Teknoloji Lideri Selçuk Bayraktar, Kana..."
2,Bilim Ve Teknoloji,"Soyuz MS-18 uzay aracı, Kazakistan’ın Baykonur..."
3,Bilim Ve Teknoloji,"Siyasetten ekonomiye, spordan sağlığa, özel ha..."
4,Bilim Ve Teknoloji,Öğrencilerin uzay teknolojileri ve roket bilim...
...,...,...
3056,Spor,UEFA Avrupa Ligi'nde 2016-2017 sezonu grup kur...
3057,Spor,Spor Toto Süper Lig'in 2. haftasında görev ala...
3058,Spor,Geçen sezonun sona ermesinin ardından Trabzons...
3059,Spor,"TFF'den yapılan açıklamada, daha önce kurulda ..."


In [4]:
a["kategori"].unique()
a['labels'] = pd.factorize(a.kategori)[0]

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train, test = train_test_split(a, test_size=0.2, random_state=42)

train=train[["metin","labels"]]
test=test[["metin","labels"]]

#for bert text = string      label = int
train["metin"]=train["metin"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

In [7]:
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-uncased', num_labels=5, use_cuda=False,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3, "train_batch_size": 64 , "fp16":False, "output_dir": "bert_model"})

Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [8]:

model.train_model(train)

result, model_outputs, wrong_predictions = model.eval_model(test)

predictions = model_outputs.argmax(axis=1)

actuals = test.labels.values

D:\Anaconda\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2448 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/39 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/39 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/39 [00:00<?, ?it/s]

D:\Anaconda\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/613 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/77 [00:00<?, ?it/s]

In [9]:
predictions[:10]

array([3, 0, 2, 2, 3, 0, 4, 4, 1, 2], dtype=int64)

In [10]:
actuals[:10]

array([3, 0, 2, 2, 3, 0, 4, 4, 1, 2], dtype=int64)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.9722675367047309

In [12]:
örnek = test.iloc[43]['metin']
print(örnek)

Kanserojen maddeler, serum ambalajlarına kadar sızdı. İstanbul’da, iki lise öğrencisi yaptığı araştırmayla bu gerçeği ortaya çıkardı. "Sağlık Endüstrilerinde Yapısal Dönüşüm Programı"yla tıbbi cihaz ve malzeme ihtiyacının yüzde 20'sinin, ilaç ihtiyacının ise yüzde 60'ının yerli üretimle karşılanması planlanıyor.


In [21]:
denemem=pd.read_excel("denemem.xlsx",names=["kategori","metin"])
örnek=denemem.iloc[13]["metin"]
tahmin=model.predict([örnek])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
if tahmin[0] ==0:
  print("BİLİM VE TEKNOLOJİ")
elif tahmin[0]==1:
  print("EKONOMİ")
elif tahmin[0]==2:
  print("SAĞLIK")
elif tahmin[0]==3:
  print("SİYASET")
else:
  print("SPOR")

SAĞLIK


In [18]:
denemem

,kategori,metin
0,Spor,İngiltere Premier Lig'in 33. haftasında Liverp...
1,Siyaset,Çanakkale İl Genel Meclisi Başkanı ve AK Parti...
2,Siyaset,Ana muhalefet partisi CHP’nin 12 yıldır başarı...
3,Siyaset,CHP’nin seçimden seçime seçmenini hatırladığın...
4,Spor,Premier Lig'de 2017-2018 ve 2018-2019 sezonlar...
5,Spor,Süper Lig'de 37. hafta mücadelesinde Konyaspor...
6,Ekonomi,ABD Ticaret Bakanı Gina Raimondo da iklim deği...
7,Ekonomi,BD'nin ev sahipliğinde 40 ülke liderinin katıl...
8,Ekonomi,"Artık birçok işletme, modellerinde gölge karbo..."
9,Teknoloji,Apple dün gerçekleştirdiği etkinlikte M1 işlem...
